<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_08_accountant_persist_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 8 — Accountant: Marketing Copy Generator + Persistent Storage

## 🔍 Темы урока
- **OpenAI Embeddings в LlamaIndex**: семантическая индексация и поиск
- **Sensitive vs Safe Tools** в LangGraph: ветвление для фильтрации
- **Persistent Storage**: сохранение индексов на диск

## 🧠 Теория с описанием API
### 🟢 OpenAI Embeddings
```python
from llama_index.embeddings.openai import OpenAIEmbedding
embedding_model = OpenAIEmbedding(model="text-embedding-3-small")
```
**Используется в:** индексации и семантическом поиске.

### 🟠 Sensitive vs Safe Tools (LangGraph)

В LangGraph концепция Sensitive vs Safe Tools предназначена для управления доступом агентов к различным инструментам в зависимости от их потенциального воздействия на систему. Это позволяет разработчикам создавать более безопасные и контролируемые рабочие процессы, особенно в сценариях, где автоматические действия могут повлиять на критические данные или процессы.

🛡️ Что такое Sensitive и Safe Tools?
Safe Tools: Инструменты, которые выполняют только операции чтения или не изменяют состояние системы. Например, поиск информации или генерация текста.

Sensitive Tools: Инструменты, которые могут изменять данные или выполнять критические операции, такие как запись в базу данных, отправка электронных писем или выполнение финансовых транзакций.

Разделение инструментов на эти категории позволяет внедрять механизмы контроля, такие как подтверждение действий человеком (human-in-the-loop), перед выполнением чувствительных операций.
LangChain

Как это реализуется в LangGraph?
LangGraph использует условные переходы (conditional edges) для маршрутизации запросов к соответствующим инструментам на основе их категории. Это достигается с помощью функции маршрутизации, которая анализирует состояние и определяет, какой инструмент должен быть вызван.
Medium

Пример функции маршрутизации:
```python
def route_request(state):
    if "цена" in state["input"]:
        return {"next": "safe_generation"}
    return {"next": "creative_generation"}
```
**Используется в:** построении безопасных сценариев генерации.

👥 Пример использования: Поддержка клиентов
В сценарии поддержки клиентов можно классифицировать действия следующим образом:

Safe Tools: Поиск информации о продуктах, ответы на часто задаваемые вопросы.

Sensitive Tools: Изменение информации о заказе, обработка возвратов, обновление учетных данных пользователя.

При использовании Sensitive Tools можно внедрить механизм подтверждения действий человеком перед их выполнением, обеспечивая дополнительный уровень безопасности.

✅ Преимущества подхода
Повышенная безопасность: Предотвращает выполнение критических операций без надлежащего контроля.

Гибкость: Позволяет агентам автоматически обрабатывать безопасные запросы, снижая нагрузку на операторов.

Прозрачность: Обеспечивает ясное разделение между различными типами операций, упрощая аудит и отладку.

### 🔵 Persistent Storage
```python
# Сохраняем:
index.storage_context.persist(persist_dir="./index_store")
# Загружаем:
from llama_index.core import load_index_from_storage
index = load_index_from_storage("./index_store")
```
**Используется в:** продакшн, кэширование, ускорение.


In [4]:
!pip install llama-index
!pip install docx2txt


In [2]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [12]:
# ✅ Импорты и загрузка данных
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import load_index_from_storage
import os
from llama_index.core import StorageContext, load_index_from_storage

docs = SimpleDirectoryReader("data").load_data()
embed_model = OpenAIEmbedding(model="text-embedding-3-small")


Это — пример кода из LlamaIndex для постоянного хранения и загрузки индекса, чтобы не пересоздавать его каждый раз:

In [6]:
# TODO: Создай индекс и сохрани его в папку ./index_store
index = VectorStoreIndex.from_documents(docs, embed_model=embed_model)
index.storage_context.persist("./index_store")


In [14]:
# TODO: Загрузи индекс из хранилища и создай query engine
storage_context = StorageContext.from_defaults(persist_dir="./index_store")
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()
print(query_engine.query("С кем был заключен договор аренды"))


The lease agreement was signed between Ivanov Ivan Sergeevich as the Landlord and Smirnov Petr Valeryevich as the Tenant.


### Sensitive vs Safe Tools (LangGraph)

🧱 1. Реализовать фильтр:
Определять, какой шаблон использовать — безопасный (safe) или креативный (full / creative) — в зависимости от содержания запроса.

✅ Пример фильтра:

In [18]:
def is_sensitive(text):
    sensitive_keywords = ["цена", "деньги", "карта", "скидка", "оплата"]
    return any(keyword in text.lower() for keyword in sensitive_keywords)


🧾 2. Создать два шаблона генерации:
🟢 Безопасный шаблон (safe_template):
Используется, если запрос содержит чувствительные слова. Выдаёт формальный/нейтральный текст.

In [19]:
safe_template = "Описание продукта: {description}. Узнайте больше на нашем сайте."


🔵 Полный шаблон (creative_template):
Используется для обычных запросов. Генерирует более рекламный/креативный текст.

In [17]:
creative_template = "🔥 Не упустите шанс! {description} — только сегодня на сайте!"


🔄 Вместе это работает так:

In [20]:
def generate_copy(description):
    template = safe_template if is_sensitive(description) else creative_template
    return {
        "email": template.format(description=description),
        "social": f"{description} — подробнее на сайте! #новинка #скидка",
        "snippet": f"{description} — узнайте больше."
    }


Это просто Python-ветвление, которое работает без LangGraph.

🧩 А при чём здесь LangGraph?
LangGraph — это графовый фреймворк для управления состоянием и маршрутизации запросов, часто используемый в архитектурах с LLM. Он позволяет делать то же самое, но декларативно, через узлы и переходы.

✅ Как LangGraph реализует такое ветвление:
Ты задаёшь узлы: safe_generation, creative_generation.

Создаёшь условный переход между ними через функцию:


In [16]:
def route_request(state):
    if "цена" in state["input"]:
        return {"next": "safe_generation"}
    return {"next": "creative_generation"}

#graph.add_conditional_edges("router", route_request)



При запуске граф сам решает, куда направить токен — к безопасному или креативному обработчику.

📊 Когда использовать LangGraph
Когда логика становится сложной (много условий, шагов, подграфов).

Когда нужен контроль за потоками, чекпоинтинг, human-in-the-loop.

В продакшене — для визуального аудита и отладки.



🔄 LangGraph Demo: Sensitive vs Safe Tools
Этот ноутбук демонстрирует, как использовать LangGraph для маршрутизации запросов между "чувствительным" и "безопасным" генератором маркетинговых текстов. Это архитектурный паттерн "Sensitive vs Safe Tools".

🧠 Что делает граф:
Запросы с чувствительными словами (цена, скидка, карта...) → идут в safe_generator.
Остальные → в creative_generator.
🧰 Используемые компоненты:
StateGraph из LangGraph
add_node() для генераторов
add_conditional_edges() для фильтрации

In [21]:
!pip install -q langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 14.0 MB/s eta 0:00:00


In [22]:
from langgraph.graph import StateGraph

In [64]:
def is_sensitive(text):
    keywords = ['цена', 'скидка', 'деньги', 'карта']
    return any(word in text.lower() for word in keywords)

def router_node(state):
    print(f"router_node: received input = {state['input']}")
    if is_sensitive(state["input"]):
        print("routing to SAFE")
        return "safe"  # 🔄 просто строка
    print("routing to CREATIVE")
    return "creative"

def creative_generator(state):
  print("CREATIVE NODE RUNNING")
  return {
      "input": state["input"],
      "output": f"CREATIVE TEXT: {state['input']}! 🔥 Не упусти шанс!"
    }

def safe_generator(state):
  return {
      "input": state["input"],
      "output": f"SAFE TEXT: {state['input']} — уточните детали на сайте."
    }



In [65]:
from langgraph.graph import StateGraph
from typing import TypedDict

class MyState(TypedDict, total=False):  # 👈 это главное
    input: str
    output: str

graph = StateGraph(state_schema=MyState)
graph.add_node("safe", safe_generator)
graph.add_node("creative", creative_generator)
graph.add_node("router", lambda state: {"input": state["input"]}) # узел прокладка
graph.add_conditional_edges("router", router_node)  # условие напрямую
graph.set_entry_point("router")
graph.set_finish_point("safe")
graph.set_finish_point("creative")

app = graph.compile()



In [66]:
result = app.invoke({"input": "Крутая кофеварка со скидкой 30%", "output": ""})
print(result["output"])



router_node: received input = Крутая кофеварка со скидкой 30%
routing to CREATIVE
CREATIVE NODE RUNNING
CREATIVE TEXT: Крутая кофеварка со скидкой 30%! 🔥 Не упусти шанс!


In [67]:
print(result.get("output", "нет вывода"))


CREATIVE TEXT: Крутая кофеварка со скидкой 30%! 🔥 Не упусти шанс!


#🛠️ Практика
Цель: Создать RAG-агента, который:
- **Принимает описание продукта.**
- **Определяет чувствительность запроса.**
- **Генерирует 3 формата текста (email, social, snippet).**
- **Использует безопасный шаблон для чувствительных тем.**
- **Сохраняет и восстанавливает индекс с помощью persist.**


Шаги:

1. **Подготовка данных: Соберите 5–10 реальных описаний продуктов и сохраните их в папке data/.**
2. **Создание и сохранение индекса:**


In [68]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

documents = SimpleDirectoryReader("data").load_data()
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
index.storage_context.persist(persist_dir="./index_store")


3. **Загрузка индекса:**

In [70]:
# TODO: Загрузи индекс из хранилища и создай query engine
storage_context = StorageContext.from_defaults(persist_dir="./index_store")
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()

4. **Фильтрация чувствительных запросов:**


In [71]:
def is_sensitive(text):
    sensitive_keywords = ["цена", "скидка", "деньги", "карта"]
    return any(keyword in text.lower() for keyword in sensitive_keywords)


5. **Шаблоны генерации:**

In [72]:
safe_template = "Описание продукта: {description}. Узнайте больше на нашем сайте."
creative_template = "Представляем {description}! Не упустите шанс — посетите наш сайт прямо сейчас!"


6. **Генерация текстов:**


In [73]:
def generate_copy(description):
    template = safe_template if is_sensitive(description) else creative_template
    email = template.format(description=description)
    social = f"{description} — подробнее на сайте! #новинка #скидка"
    snippet = f"{description} — узнайте больше."
    return {"email": email, "social": social, "snippet": snippet}


7. **Генерация текстов:**

In [74]:
test_description = "Новый смартфон с отличной камерой и скидкой 20%"
copies = generate_copy(test_description)
for format, text in copies.items():
    print(f"{format.capitalize()}:\n{text}\n")


Email:
Представляем Новый смартфон с отличной камерой и скидкой 20%! Не упустите шанс — посетите наш сайт прямо сейчас!

Social:
Новый смартфон с отличной камерой и скидкой 20% — подробнее на сайте! #новинка #скидка

Snippet:
Новый смартфон с отличной камерой и скидкой 20% — узнайте больше.

